# EDA

## Setup dependencies and config

In [1]:
import datetime
import matplotlib.pyplot as plt
import altair as alt
import dateutil

import numpy as np
from collections import Counter
import json
from pathlib import Path
import pandas as pd
from tqdm import tqdm

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [2]:
df = pd.read_pickle('df.pkl')

In [3]:
df

,scrapedAt,playlist,id,title,channel,duration,rank,hour,day
0,2021-11-12 23:00:20.735000+00:00,News / Nachrichten / PL3ZQ5CpNulQnRmIg0qmrmA-Q...,gZD4hH8l3AA,AFP Deutschland - Niederlande kündigen Lockdow...,AFP Deutschland,45000,0,2021-11-12 23:00:00+00:00,2021-11-12 00:00:00+00:00
1,2021-11-12 23:00:20.735000+00:00,News / Nachrichten / PL3ZQ5CpNulQnRmIg0qmrmA-Q...,YBZ_GmnA7e8,Handelsblatt - Klimagipfel neigt sich dem Ende...,Handelsblatt,116000,1,2021-11-12 23:00:00+00:00,2021-11-12 00:00:00+00:00
2,2021-11-12 23:00:20.735000+00:00,News / Nachrichten / PL3ZQ5CpNulQnRmIg0qmrmA-Q...,erJgwCwk384,BILD - Putin schickt Atombomber nach Belarus –...,BILD,102000,2,2021-11-12 23:00:00+00:00,2021-11-12 00:00:00+00:00
3,2021-11-12 23:00:20.735000+00:00,News / Nachrichten / PL3ZQ5CpNulQnRmIg0qmrmA-Q...,z0lbTAewCjM,münchen.tv - Das sagen die Münchner Wirte zur ...,münchen.tv,126000,3,2021-11-12 23:00:00+00:00,2021-11-12 00:00:00+00:00
4,2021-11-12 23:00:20.735000+00:00,News / Nachrichten / PL3ZQ5CpNulQnRmIg0qmrmA-Q...,OEL1lYd5aKE,WELT Nachrichtensender - AKTUELLE CORONA-SCHOC...,WELT Nachrichtensender,59000,4,2021-11-12 23:00:00+00:00,2021-11-12 00:00:00+00:00
...,...,...,...,...,...,...,...,...,...
816447,2021-12-16 15:44:30.555000+00:00,Health News / Gesundheitsnachrichten / PLG3wws...,XUsX8MUlvjQ,DW Deutsch - 40 Jahre HIV/AIDS und noch immer ...,DW Deutsch,438000,2,2021-12-16 15:00:00+00:00,2021-12-16 00:00:00+00:00
816448,2021-12-16 15:44:30.555000+00:00,Health News / Gesundheitsnachrichten / PLG3wws...,mvUH-zJYHxA,tagesschau - Weltaidstag: Corona-Pandemie wirf...,tagesschau,566000,3,2021-12-16 15:00:00+00:00,2021-12-16 00:00:00+00:00
816449,2021-12-16 15:44:30.555000+00:00,Health News / Gesundheitsnachrichten / PLG3wws...,4v3nL7gBshA,faz - Videografik: So attackiert HIV das mensc...,faz,77000,4,2021-12-16 15:00:00+00:00,2021-12-16 00:00:00+00:00
816450,2021-12-16 15:44:30.555000+00:00,Health News / Gesundheitsnachrichten / PLG3wws...,CeWXEMPLXNw,DER SPIEGEL - Recherche auf Corona-Intensivsta...,DER SPIEGEL,340000,5,2021-12-16 15:00:00+00:00,2021-12-16 00:00:00+00:00


In [4]:
min_day = df['day'].min().to_pydatetime()
max_day = df['day'].max().to_pydatetime()
diff_days = (max_day-min_day).days

## Load complete video information 

In [5]:
data = []
for p in tqdm(list(Path().glob('channeldata/*.json')), desc='Load channel data'):
    d = json.loads(p.read_text())
    data.extend(map(json.loads, d))

Load channel data: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 59/59 [00:04<00:00, 12.79it/s]


In [6]:
chan_data = []
for d in data:
    parsed_date = dateutil.parser.isoparse(d['contentDetails']['videoPublishedAt'])
    # exclude last day
    if min_day <= parsed_date < max_day:
#     if min_day <= parsed_date < (max_day - datetime.timedelta(days=10)):
        chan_data.append((d['snippet']['channelTitle'], d['snippet']['title'], d['snippet']['resourceId']['videoId'], parsed_date))

In [7]:
df_chan = pd.DataFrame(data=chan_data, columns = [ 'channel', 'title', 'id', 'publishedAt'])
uniq_ids = set(df['id'].unique())
df_chan['hit'] = df_chan['id'].apply(lambda x: x in uniq_ids)
df_chan

,channel,title,id,publishedAt,hit
0,hrfernsehen,Überfall während Spendenlauf I maintower,Wk_V9dY7b_g,2021-12-15 23:04:27+00:00,False
1,hrfernsehen,Borussia Mönchengladbach - Eintracht Frankfurt...,YmGs1_1G0X8,2021-12-15 23:00:46+00:00,False
2,hrfernsehen,7 Tage... in der Hartz IV-Maßnahme | dokus und...,JbM6tMl7dMg,2021-12-15 17:30:13+00:00,False
3,hrfernsehen,Lichterlabyrinth in Frankfurt – Zur Ruhe komme...,TbG-lb-e8hQ,2021-12-15 17:29:53+00:00,False
4,hrfernsehen,Zweiter Corona-Winter: Ist mein Ski-Urlaub in ...,r_01VIzDMlw,2021-12-15 16:59:53+00:00,False
...,...,...,...,...,...
9425,Weltspiegel,Kongo: Wie Dörfer gegen die Holzindustrie kämpfen,5B0-PeDxpSM,2021-11-09 14:00:04+00:00,False
9426,Weltspiegel,Wasserstoff: Utopie oder Rettung? | Weltspiege...,jTPNPdDqUPc,2021-11-06 11:15:04+00:00,False
9427,Weltspiegel,COP26: Wie CO2-Sünder den Klimawandel stoppen ...,mLeYQvbpZOE,2021-11-05 18:00:34+00:00,True
9428,Weltspiegel,Zerstörte Heimat: Vulkanausbruch auf La Palma,DaBuq-xHShM,2021-11-03 12:00:12+00:00,False


In [8]:
len(uniq_ids)

3247

In [9]:
vis_data = []

for name, group_df in df_chan.groupby('channel'):
#     print(name)
    vcs = group_df['hit'].value_counts()
    hit = 0
    if True in vcs:
        hit = vcs[True]
    nohit = vcs[False]

    vis_data.append((name, nohit, 'not_in_playlist'))
    vis_data.append((name, hit , 'in_playlist'))

In [10]:
# https://stackoverflow.com/a/61157091/4028896

In [40]:
vis_data

[('1. FC Kaiserslautern', 19, 'not_in_playlist'),
 ('1. FC Kaiserslautern', 8, 'in_playlist'),
 ('1. FC Nürnberg - CLUB TV', 21, 'not_in_playlist'),
 ('1. FC Nürnberg - CLUB TV', 11, 'in_playlist'),
 ('1. FC Union Berlin', 28, 'not_in_playlist'),
 ('1. FC Union Berlin', 13, 'in_playlist'),
 ('ADAC', 50, 'not_in_playlist'),
 ('ADAC', 6, 'in_playlist'),
 ('AFP Deutschland', 337, 'not_in_playlist'),
 ('AFP Deutschland', 322, 'in_playlist'),
 ('ARD', 19, 'not_in_playlist'),
 ('ARD', 1, 'in_playlist'),
 ('Abendblatt-TV', 30, 'not_in_playlist'),
 ('Abendblatt-TV', 14, 'in_playlist'),
 ('Augsburger Allgemeine', 42, 'not_in_playlist'),
 ('Augsburger Allgemeine', 5, 'in_playlist'),
 ('BILD', 411, 'not_in_playlist'),
 ('BILD', 179, 'in_playlist'),
 ('BR24', 247, 'not_in_playlist'),
 ('BR24', 94, 'in_playlist'),
 ('Borussia Mönchengladbach', 20, 'not_in_playlist'),
 ('Borussia Mönchengladbach', 5, 'in_playlist'),
 ('DER AKTIONÄR TV', 243, 'not_in_playlist'),
 ('DER AKTIONÄR TV', 71, 'in_playlist'

In [11]:
alt.Chart(pd.DataFrame(data=vis_data, columns=['name', 'count', 'hit'])).transform_calculate(
    key="datum.hit == 'hit'"
).transform_joinaggregate(
    sort_key="argmax(key)", groupby=['name']
).transform_calculate(
    sort_val='datum.sort_key.count'  
).mark_bar().encode(
    x='count:Q',
    y=alt.Y('hit:O', axis=alt.Axis(title=None)),
    color=alt.Color('hit:N', legend=None),
    row=alt.Row('name:N',
        title="",
        header=alt.Header(labelAngle=0, labelAlign='left'),
        sort=alt.SortField("sort_val", order="descending")
    ), 
)

alt.Chart(...)

## Looking at WELT videos in detail

What are the difference in videos that appeared in the news playlists and those that didn't appear there?

In [12]:
df_welt = df_chan[df_chan['channel'] == 'WELT Nachrichtensender']

In [13]:
welt_hits = df_welt[df_welt['hit']]['title'].tolist()

In [14]:
welt_nohits = df_welt[df_welt['hit'] == False]['title'].tolist()

In [15]:
c_hits = Counter()
c_nohits = Counter()

for x in welt_hits:
    c_hits.update(x.split())
    
for x in welt_nohits:
    c_nohits.update(x.split())

In [16]:
c_hits.most_common(40)

[('-', 237),
 ('WELT', 223),
 ('|', 188),
 ('in', 166),
 ('für', 98),
 ('der', 98),
 ('die', 95),
 ('und', 94),
 ('von', 69),
 ('I', 67),
 ('auf', 66),
 ('CORONA:', 66),
 ('bei', 63),
 ('mit', 46),
 ('Thema', 44),
 ('Newsstream', 40),
 ('den', 39),
 ('gegen', 37),
 ('zu', 36),
 ('an', 36),
 ('News', 35),
 ('dabei', 35),
 ('IN', 35),
 ('Live', 34),
 ('ist', 34),
 ('CORONA-ZAHLEN:', 33),
 ('Inzidenz', 33),
 ('vor', 32),
 ('RKI', 32),
 ('im', 31),
 ('Deutschland', 31),
 ('AKTUELLE', 30),
 ('sich', 30),
 ('Interview', 29),
 ('–', 26),
 ('aus', 26),
 ('wird', 25),
 ('ein', 25),
 ('will', 23),
 ('nach', 23)]

In [17]:
c_nohits.most_common(40)

[('WELT', 265),
 ('|', 218),
 ('-', 202),
 ('in', 141),
 ('der', 126),
 ('I', 102),
 ('für', 99),
 ('die', 86),
 ('und', 81),
 ('–', 80),
 ('CORONA:', 80),
 ('Interview', 76),
 ('Die', 70),
 ('auf', 61),
 ('den', 60),
 ('ist', 54),
 ('News', 51),
 ('von', 51),
 ('erklärt', 50),
 ('im', 49),
 ('90', 46),
 ('SEKUNDEN', 45),
 ('aktuellen', 43),
 ('INTERVIEW', 40),
 ('CORONA', 39),
 ('IN', 38),
 ('sich', 37),
 ('an', 37),
 ('zu', 37),
 ('mit', 34),
 ('nicht', 33),
 ('vor', 30),
 ('Deutschland', 29),
 ('bei', 29),
 ('wird', 27),
 ('Sie', 25),
 ('&', 25),
 ('das', 23),
 ('Corona-Zahlen', 23),
 ('Thema', 21)]

In [18]:
import isodate

In [19]:
import pyyoutube
API_KEY = Path("secrets.txt").read_text()

api = pyyoutube.Api(api_key=API_KEY)

In [20]:
for x in tqdm(df_welt['id'].tolist()):
    fn = Path('weltdata') / (x + '.json')
    if fn.is_file():
        continue
    video_by_id = api.get_video_by_id(video_id=x)
    fn.write_text(json.dumps(video_by_id.to_dict()))

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1498/1498 [00:00<00:00, 17573.69it/s]


In [21]:
weltdata = {}
for p in tqdm(list(Path().glob('weltdata/*.json')), desc='Load welt data'):
    d = json.loads(p.read_text())
    weltdata[p.stem] = d

Load welt data: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1498/1498 [00:00<00:00, 2595.21it/s]


In [22]:
def add_info(x):
    stats = weltdata[x['id']]['items'][0]['statistics']
    x['view_count'] = stats['viewCount'] or 0
    x['like_count'] = stats['likeCount'] or 0
    x['comment_count'] = stats['commentCount'] or 0
    x['duration'] =  weltdata[x['id']]['items'][0]['contentDetails']['duration']
    return x

In [23]:
df_welt = df_welt.apply(add_info, axis=1)

In [24]:
df_welt['duration'] = df_welt['duration'].apply(isodate.parse_duration)

In [25]:
df_welt['duration'] = df_welt['duration']/np.timedelta64(1, 's')

In [26]:
df_welt['hour'] = df_welt['publishedAt'].dt.hour

In [27]:
df_welt['view_count'] = df_welt['view_count'].astype(int)
df_welt['like_count'] = df_welt['like_count'].astype(int)
df_welt['dow'] = df_welt['publishedAt'].dt.day_name()

df_welt['like_ratio'] = df_welt['like_count'] / df_welt['view_count']

In [28]:
df_welt

,channel,title,id,publishedAt,hit,view_count,like_count,comment_count,duration,hour,dow,like_ratio
1902,WELT Nachrichtensender,"ÄRZTEPRÄSIDENT REINHARDT: ""Rechnungen von Laut...",H5586ixYhoM,2021-12-15 21:15:01+00:00,True,11949,71,0,405.0,21,Wednesday,0.005942
1903,WELT Nachrichtensender,SIR LEWIS: Formel-1-Rekordweltmeister Hamilton...,oy0yM18Mkpc,2021-12-15 20:45:01+00:00,True,9939,176,0,58.0,20,Wednesday,0.017708
1904,WELT Nachrichtensender,Demokratie-Studie: Wie robust hat sich Demokra...,EqGS5saA2eM,2021-12-15 20:15:01+00:00,False,3804,52,0,308.0,20,Wednesday,0.013670
1905,WELT Nachrichtensender,TIERGARTENMORD: Baerbock weist russische Botsc...,TkGTP47ZbQY,2021-12-15 19:45:01+00:00,False,38282,400,0,301.0,19,Wednesday,0.010449
1906,WELT Nachrichtensender,"CORONA: Covid19-Impfstoffmangel! ""Das ist ein ...",spP1B2YxxWE,2021-12-15 19:15:01+00:00,False,8506,80,0,405.0,19,Wednesday,0.009405
...,...,...,...,...,...,...,...,...,...,...,...,...
3395,WELT Nachrichtensender,G20-TREFFEN IN ITALIEN: Der letzte Gipfel für ...,BA6X0I9BJNs,2021-10-29 06:45:01+00:00,False,5207,66,0,113.0,6,Friday,0.012675
3396,WELT Nachrichtensender,CORONA: Covid19-Fallzahlen steigen rasant! Jet...,wNMtf8zw8ho,2021-10-29 06:15:03+00:00,False,47192,194,0,67.0,6,Friday,0.004111
3397,WELT Nachrichtensender,FACEBOOK: META! Der Internet-Gigant von Mark Z...,T018OiJv2PE,2021-10-29 05:47:20+00:00,False,4877,53,0,57.0,5,Friday,0.010867
3398,WELT Nachrichtensender,"AKTUELLE CORONA-WELLE: Klaus Stöhr - ""Die Kran...",KQjigF96Qmw,2021-10-29 05:30:01+00:00,False,12165,137,0,332.0,5,Friday,0.011262


In [29]:
def plot_counts_hour(df, col, title, max_bars=None):
    counts = df[col].value_counts().reset_index().rename(columns={col:'counts', 'index': col})
    
    new_data = []
    for i in range(0, 24):
        if counts.loc[counts[col] == i].shape[0] == 0:
            new_data.append({'counts': 0, col: i})
    counts = counts.append(pd.DataFrame(new_data))
    
    if max_bars is not None:
        counts=counts[:max_bars]
    
    chart = alt.Chart(counts).mark_bar().encode(
        x='counts',
        y=alt.Y(col + ':N',  axis=alt.Axis(labelLimit=500))
#         y=alt.Y(col + ':N', sort='-x',  axis=alt.Axis(labelLimit=500))
    ).properties(height=400, title=title)
    return chart

def plot_counts_dow(df, col, title, max_bars=None):
    counts = df[col].value_counts().reset_index().rename(columns={col:'counts', 'index': col})
    
#     new_data = []
#     for i in range(0, 24):
#         if counts.loc[counts[col] == i].shape[0] == 0:
#             new_data.append({'counts': 0, col: i})
#     counts = counts.append(pd.DataFrame(new_data))
    
    if max_bars is not None:
        counts=counts[:max_bars]
    
    chart = alt.Chart(counts).mark_bar().encode(
        x='counts',
        y=alt.Y(col + ':N', sort=['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday'], axis=alt.Axis(labelLimit=500))
#         y=alt.Y(col + ':N', sort='-x',  axis=alt.Axis(labelLimit=500))
    ).properties(height=200, title=title)
    return chart

In [30]:
def plot_counts_view(df, col, title, max_bars=None):
    df = df[df[col] < 500000]
    
    counts = df[col].reset_index()
    if max_bars is not None:
        counts=counts[:max_bars]
    
    chart = alt.Chart(counts).mark_bar().encode(
        alt.X(col + ":Q", bin=alt.Bin(step=5000,  extent=[0, 400000])),
        y='count()',
    ).properties(height=300, width=700, title=title)
    return chart

def plot_counts_like(df, col, title, max_bars=None):
    df = df[df[col] < 20000]
    
    counts = df[col].reset_index()
    if max_bars is not None:
        counts=counts[:max_bars]
    
    chart = alt.Chart(counts).mark_bar().encode(
        alt.X(col + ":Q", bin=alt.Bin(step=100, extent=[0, 8000])),
        y='count()',
    ).properties(height=300, width=700, title=title)
    return chart

def plot_counts_duration(df, col, title, max_bars=None):
    df = df[df[col] < 6000]
    
    counts = df[col].reset_index()
    if max_bars is not None:
        counts=counts[:max_bars]
    
    chart = alt.Chart(counts).mark_bar().encode(
        alt.X(col + ":Q", bin=alt.Bin(step=100, extent=[0, 6000])),
        y='count()',
    ).properties(height=300, width=700, title=title)
    return chart

def plot_like_ratio(df, col, title, max_bars=None):
    df = df[df[col] < 6000]
    
    counts = df[col].reset_index()
    if max_bars is not None:
        counts=counts[:max_bars]
    
    chart = alt.Chart(counts).mark_bar().encode(
        alt.X(col + ":Q", bin=alt.Bin(step=0.0005, extent=[0, 0.05])),
        y='count()',
    ).properties(height=300, width=700, title=title)
    return chart

In [31]:
df_welt_hits = df_welt[df_welt['hit']]

In [32]:
df_welt_hits

,channel,title,id,publishedAt,hit,view_count,like_count,comment_count,duration,hour,dow,like_ratio
1902,WELT Nachrichtensender,"ÄRZTEPRÄSIDENT REINHARDT: ""Rechnungen von Laut...",H5586ixYhoM,2021-12-15 21:15:01+00:00,True,11949,71,0,405.0,21,Wednesday,0.005942
1903,WELT Nachrichtensender,SIR LEWIS: Formel-1-Rekordweltmeister Hamilton...,oy0yM18Mkpc,2021-12-15 20:45:01+00:00,True,9939,176,0,58.0,20,Wednesday,0.017708
1908,WELT Nachrichtensender,"SAHRA WAGENKNECHT: ""Impfdruck als Dank"" – ""Rie...",Yd1xDFuQ6m0,2021-12-15 18:15:01+00:00,True,522110,14554,0,447.0,18,Wednesday,0.027875
1910,WELT Nachrichtensender,TESLA: Nach tödlichem Unfall! Größtes Taxi-Unt...,8rl-ldSG57k,2021-12-15 17:15:01+00:00,True,17489,172,0,52.0,17,Wednesday,0.009835
1913,WELT Nachrichtensender,AFD: Zoff über Innenausschuss! Parteien laufen...,jkQQE53ljTE,2021-12-15 16:15:01+00:00,True,29960,225,0,169.0,16,Wednesday,0.007510
...,...,...,...,...,...,...,...,...,...,...,...,...
3377,WELT Nachrichtensender,HOCHSPANNUNG VOR G20-GIPFEL: Extreme Sicherhei...,fK1WnwrnKoc,2021-10-29 14:00:20+00:00,True,10936,140,0,355.0,14,Friday,0.012802
3378,WELT Nachrichtensender,OLAF SCHOLZ: Statement zu Vorbereitungen auf G...,SFZL8R33-sw,2021-10-29 13:48:09+00:00,True,12909,187,0,520.0,13,Friday,0.014486
3381,WELT Nachrichtensender,SCHULDEN ODER SPAREN: Gegenpole - Lindner und ...,oLZV-FYFEDw,2021-10-29 12:30:10+00:00,True,7581,86,0,119.0,12,Friday,0.011344
3393,WELT Nachrichtensender,AMPEL-PARTEIEN: Jetzt geht es ans Eigemachte! ...,XVMzqmtP9gw,2021-10-29 07:45:01+00:00,True,29203,202,0,116.0,7,Friday,0.006917


In [33]:
df_welt_nohits = df_welt[df_welt['hit'] == False]

In [34]:
alt.hconcat(*[plot_counts_hour(df_welt_hits, 'hour', 'in playlist: hour'), plot_counts_hour(df_welt_nohits, 'hour', 'not in playlist: hour')])

alt.HConcatChart(...)

In [35]:
alt.hconcat(*[plot_counts_dow(df_welt_hits, 'dow', 'in playlist: day of week'), plot_counts_dow(df_welt_nohits, 'dow', 'not in playlist: day of week')])

alt.HConcatChart(...)

In [36]:
alt.vconcat(*[plot_counts_view(df_welt_hits, 'view_count', 'in playlist: view_count'), plot_counts_view(df_welt_nohits, 'view_count', 'not in playlist: view_count')])

alt.VConcatChart(...)

In [37]:
alt.vconcat(*[plot_counts_like(df_welt_hits, 'like_count', 'in playlist: like_count'), plot_counts_like(df_welt_nohits, 'like_count', 'not in playlist: like_count')])

alt.VConcatChart(...)

In [38]:
alt.vconcat(*[plot_counts_duration(df_welt_hits, 'duration', 'in playlist: duration (in seconds)'), plot_counts_duration(df_welt_nohits, 'duration', 'not in playlist: duration (in seconds)')])

alt.VConcatChart(...)

In [39]:
alt.vconcat(*[plot_like_ratio(df_welt_hits, 'like_ratio', 'in playlist: like ratio (the higher the more likes per view)'), plot_like_ratio(df_welt_nohits, 'like_ratio', 'not in playlist: like ratio (the higher the more likes per view)')])

alt.VConcatChart(...)